In [1]:
import numpy as np

In [2]:
beamType = 'DoppiaRastremazione'

# beamType = 'Curva'
# beamType = 'Centinata'

timberType = 'lamellare'
# timberType = 'massiccio'

In [48]:
# N/mm2
f_mk = 28
f_vk = 3.5
f_c90k = 2.5
f_t90k = 0.5
E0mean = 12600
E005 = 10500
Gmean = 650

gammaM = 1.45
kmod = 0.9

alpha = 3.5 # in gradi °
b = 240
h_ap = 1800-500 # altezza apice
h_0 = 1315-500 # altezza appoggio
l = 16000 

alpha = np.radians(alpha)

In [4]:
# volume della trave 
V_b = (h_ap + h_0) * l/2 * b 
V_b = V_b/10**9 # da mm3 a m3
print('V_b:', V_b, 'm3')

# vale solo per la trave a doppia rastremazione:
# volume della zona di colmo
# formula alternativa: b * h_ap**2 * (1 - 0.25 * np.tan(alpha)) / 10**9
h_ap_05 = h_ap - np.tan(alpha)*0.5*h_ap
V_colmo =  (h_ap_05 + h_ap) * 0.5*h_ap * b
V_colmo = V_colmo/10**9
print('V_colmo:', V_colmo, 'm3')

# volume zona di colmo per la normativa
V = min(2/3 * V_b , V_colmo)
print('2/3 V_b:', 2/3 * V_b, 'm3')
print('V:', V, 'm3')

V_0 = 0.01

V_b: 4.0608 m3
V_colmo: 0.3993981103167409 m3
2/3 V_b: 2.7072000000000003 m3
V: 0.3993981103167409 m3


In [5]:
M_d = 641.5 #kNm
V_d = 158.4 #kN

M_d = M_d * 10**6 
V_d = V_d * 10**3 

HP: $ \sigma_{m,\alpha,d} = \sigma_{m,0,d}$

In [6]:
sigma_m_alpha_d = 6 * M_d / (b*h_ap**2) # nel punto in mezzeria
sigma_m_alpha_d

9.4896449704142

# Parte di trave a doppia rastremazione
Capitolo 6.4.3 EC5

## 1. Flessione

In [7]:
f_md = kmod * f_mk / gammaM
f_md

17.379310344827587

da ricontrollare i k

In [8]:
t = 30 # spessore lamelle LVL
r_in = 17000 #raggio interno se curva o centinata. altrimenti non serve
if beamType == 'DoppiaRastremazione':
    k_r = 1
else:
    k_r = 1 if r_in/t >240 else 0.76 + 0.001 * r_in/t
    # t spessore lamelle 
k_r    

1

In [9]:
alpha_ap = np.radians(3.5) # vedi la figura
if beamType == 'DoppiaRastremazione':
    k_l = k_1 = 1 + 1.4*np.tan(alpha_ap) + 5.4*np.tan(alpha_ap)**2
    print('k_l: ', k_l)
else:
    k_1 = 1 + 1.4*np.tan(alpha_ap) + 5.4*np.tan(alpha_ap)**2
    k_2 = 0.35 - 8*np.tan(alpha_ap)
    k_3 = 0.6 + 8.3*np.tan(alpha_ap) - 7.8*np.tan(alpha_ap)**2
    k_4 = 6*np.tan(alpha_ap)**2
    r = r_in + 0.5 * h_ap
    k_l = k_1 + k_2*(h_ap/r) + k_3*(h_ap/r)**2 + k_4*(h_ap/r)**3
    print(k_1)
    print(k_2)
    print(k_3)
    print(k_4)
    print(r)
    print('k_l: ', k_l)

sigma_md = k_l * 6 * M_d / (b*h_ap**2) # nel punto in mezzeria

print('sigma_md: ', sigma_md)
print('f_md: ', f_md)
sigma_md < k_r *f_md


k_l:  1.105828345170509
sigma_md:  10.493918393888782
f_md:  17.379310344827587


True

## 2. Trazione 90 all'apice

In [10]:
k_dis =  1.4 if beamType == 'DoppiaRastremazione' or  beamType == 'Curva' else  1.7
print('k_dis:', k_dis)

k_vol = 1 if timberType == 'massiccio' else (V_0/V)**0.2 
print('k_vol:', k_vol)

k_dis: 1.4
k_vol: 0.478320284636461


In [11]:
f_t90d = f_t90k * kmod / gammaM
f_t90d

0.3103448275862069

In [12]:
k_dis * k_vol * f_t90d

0.20782191677308304

In [13]:
# solo per trave doppia rastremazione. aggiungere le altre!
k_p = 0.2 * np.tan(alpha) 
k_p

0.012232524030096862

In [14]:
sigma_t90d_ap = 6 * M_d / (b*h_ap**2) # M all'apice!
sigma_t90d_ap

9.4896449704142

In [15]:
k_p * sigma_t90d_ap

0.11608231013767954

In [16]:
print('S:', k_p * sigma_t90d_ap)
print('R:', k_dis * k_vol * f_t90d)

k_p * sigma_t90d_ap < k_dis * k_vol * f_t90d

S: 0.11608231013767954
R: 0.20782191677308304


True

## 3. Taglio + Trazione a $0.5\cdot h_{ap}$ 

In [17]:
print('x di 0.5*h_ap: ', l/2 - 0.5*h_ap) 
print('h trave a 0.5*h_ap: ', h_ap_05)

# Mettere i valori a tale x
M = 636.2 * 10**6
V = 14.6 * 10**3

x di 0.5*h_ap:  7350.0
h trave a 0.5*h_ap:  1260.2442969021852


In [18]:
f_vd = f_vk * kmod / gammaM
print('f_vd:', f_vd)

b_eff = 2.5/f_vk * b
print('b_eff:', b_eff)

tau_d_05ap = 1.5*V/(b_eff*h_ap_05)
#tau_d_05ap = 1.5*V/(0.67*b*h_ap_05)
tau_d_05ap

f_vd: 2.1724137931034484
b_eff: 171.42857142857144


0.10136923476981655

In [19]:
def sigma_t90d(M_d,b,h,k_p):
    sigma_t90d = 6 * M_d / (b*h**2) * k_p
    print('M_d: ', M_d)
    print('b: ', b)
    print('h: ', h)
    print('k_p: ', k_p)
    print('sigma_t90d: ', sigma_t90d)
    return sigma_t90d

sigma_t90d_05ap = sigma_t90d(M , b , h_ap_05 , k_p)

M_d:  636200000.0
b:  240
h:  1260.2442969021852
k_p:  0.012232524030096862
sigma_t90d:  0.12250117926934026


In [20]:
tau_d_05ap/f_vd + sigma_t90d_05ap/(f_t90d * k_dis * k_vol) 

0.636114677256977

In [21]:
print('k_dis: ', k_dis)
print('k_vol: ', k_vol)

tau_d_05ap/f_vd + sigma_t90d_05ap/(f_t90d * k_dis * k_vol) < 1

k_dis:  1.4
k_vol:  0.478320284636461


True

# Parte di trave a rastremazione semplice
Capitolo 6.4.2 EC5. Fino a $0.5 h_{ap}$ dall'asse di simmetria

In [22]:
M_max = 60.6*10**6 # M corrispondente alla x_max

Q = 19.5595 #kN/m

In [45]:
#V_d * x_max - Q*x_max**2 / 2

548436281.4201183

In [24]:
# Solo se trave simmetrica e carico distribuito!
x_max = l * h_0 / (2*h_ap)
print('x_max: ', x_max)

h_max = h_ap - np.tan(alpha)*(l/2 -x_max) # altezza corrispondente alla x_max
print('h_max: ', h_max)

#sigma_md_max = 6*M_max / (b*h_max**2)
sigma_md_max = 0.75*Q*l**2/b * 1/(h_0*(2*h_ap-h_0))
print('sigma_md_max: ', sigma_md_max)

x_max:  5015.384615384615
h_max:  1117.4531029354775
sigma_md_max:  10.756027564399993


In [25]:
f_c90d = f_c90k * kmod / gammaM
f_c90d

1.5517241379310345

### 5. Bordo inclinato (compresso)

calcolare l'altro k_m se il momento va dall'altra parte e quindi è teso

In [26]:
k_mAlpha_compressione = 1 / np.sqrt(1 + (np.tan(alpha)*f_md/(1.5*f_vd))**2 + ((np.tan(alpha))**2*f_md/f_c90d)**2 )
k_mAlpha_compressione

0.9499446894142974

In [27]:
k_mAlpha_compressione * f_md

16.50938356775193

In [28]:
sigma_md_max < k_mAlpha_compressione * f_md

True

### 6. Bordo non inclinato (teso)

In [29]:
print('sigma_md_max: ', sigma_md_max)
print('f_md: ', f_md)

sigma_md_max < f_md

sigma_md_max:  10.756027564399993
f_md:  17.379310344827587


True

## 4. Stabilità flessotorsionale al colmo

In [30]:
# Vedi pagina 291 del libro
h=h_ap

l_t = l # distanza tra due ritegni torsionali successivi 
a1 = 1.13
a2 = 1.44
az = h/2

# caso generico per il RETTANGOLO e non semplificato per h/b > 4
B = E0mean * b**3 * h / 12 # rigidezza flessionale attorno asse z
T = Gmean * b**3 * h / (3*(1 + 0.6 * b/h)) # rigidezza torsionale 

#l_eff = l_t / (a1 * (1 - a2 * az/l_t * np.sqrt(B/T)))
l_eff = 0.9*l

sigma_mcrit = np.pi/l_eff * b**2 / h * E005 * np.sqrt(Gmean/E0mean)
lambda_relm = np.sqrt(f_mk/sigma_mcrit)
if lambda_relm < 0.75:
    k_crit = 1
elif lambda_relm < 1.14:
    k_crit = 1.56 - .75*lambda_relm
else:
    k_crit = 1/(lambda_relm**2)

print('l_eff: ', l_eff)
print('l_eff/l: ', l_eff/l)
print('sigma_mcrit: ', sigma_mcrit)
print('lambda_relm: ', lambda_relm)
print('k_crit: ', k_crit)

l_eff:  14400.0
l_eff/l:  0.9
sigma_mcrit:  23.05298759340608
lambda_relm:  1.1020858017945503
k_crit:  0.7334356486540874


In [31]:
k_crit * f_md

12.746605755919312

In [32]:
sigma_md

10.493918393888782

In [33]:
sigma_md < k_crit * f_md #sigma_md con il k_l ?

True

## 7. Taglio all'appoggio

In [34]:
b_eff = 2.5/f_vk * b
print('b_eff:', b_eff)

tau_d_appoggio = 1.5*V_d/(b_eff*h_0)
print('tau_d_appoggio:', tau_d_appoggio)

print('f_vd:', f_vd)

b_eff: 171.42857142857144
tau_d_appoggio: 1.700613496932515
f_vd: 2.1724137931034484


In [35]:
tau_d_appoggio < f_vd

True

## 8. Compressione 90 appoggio

In [36]:
l_appoggio = 300
A_ef = b* (l_appoggio+30+0) # min(30mm , a , l_appoggio, l/2) sx e dx
k_c90 = 1.75

sigma_c90_appoggio = V_d / A_ef
print('A_ef:', A_ef)
print('sigma_c90_appoggio:', sigma_c90_appoggio)

A_ef: 79200
sigma_c90_appoggio: 2.0


In [37]:
print('R:', k_c90*f_c90d)
sigma_c90_appoggio < k_c90 * f_c90d

R: 2.7155172413793105


True

In [38]:
220

220

# 9. Freccia

In [39]:
q = [8.6428,10.9167]

k_m = (h_0/h_ap)**3 * 1/(0.15+0.85*(h_0/h_ap))
print('k_m: ', k_m)
k_v = 2/(1 + (h_ap/h_0)**(2/3))
print('k_k_vm: ', k_v)

#Nella sezone di appoggio e poi vengono usati i due coefficienti
J_0 = b*h_0**3 / 12
print('J_0:', J_0/10**9, '* 10**9 mm4')
print('\n')

chi = 1.2
def w_inst(q):
    w_inst_M = k_m * 5/384 * q * l**4 / (E0mean * J_0) 
    w_inst_V = chi * 1/8 * q * l**2 / (Gmean * b * h_0)
    w_inst = w_inst_M + w_inst_V
    return w_inst

w=[]
for qq in q:
    w_i = w_inst(qq)
    w.append(w_i)
    print(w_i)
    print(l/w_i)
    print('\n')

print('w_inst totale = ',np.sum(w))
print(l/250)
print(l/300)
print(l/500)

k_m:  0.3608240199027193
k_k_vm:  0.8456009531060444
J_0: 10.8268675 * 10**9 mm4


22.11759244206245
723.4060416798245


27.936678091852546
572.7237843881746


w_inst totale =  50.054270533914995
64.0
53.333333333333336
32.0


In [40]:
l/w_inst(Q)

319.6530451714199

In [47]:
k_def = 0.6
psi2 = 0.

w_netFin_1 = w[0]*(1 + k_def) 
print('w_netFin_1: ', w_netFin_1)

w_netFin_2 = w[1]*(1 + k_def*psi2)
print('w_netFin_2: ', w_netFin_2)

w_netFin_tot = w_netFin_1+w_netFin_2
print('w_netFin_tot: ', w_netFin_tot)
print('l/w_netFin_tot: ', l/w_netFin_tot)

w_netFin_1:  35.38814790729992
w_netFin_2:  27.936678091852546
w_netFin_tot:  63.324825999152466
l/w_netFin_tot:  252.66551857898736
